In [1]:
import requests
import os
import pandas as pd

In [2]:
df=pd.read_csv('data.csv')
df.head()

,name,latitude,longitude,free_bikes,empty_slots,total_bike_cap
0,East End Library,37.539779,-77.411460,0,6,6
1,Sydney Park,37.546968,-77.456599,0,16,16
2,Broad & Harrison,37.551472,-77.451937,0,19,19
3,Broad & Lombardy,37.553716,-77.455681,0,23,23
4,Broad Rock Library,37.483291,-77.479390,0,10,10


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
api_key = os.environ['foursquareapi']
llatitude = ""
longitude = ''
radius = str(1000)
categories = '11124,13003'
headers = {"Accept": "application/json"}
headers['Authorization'] = api_key
foursquaredata = pd.DataFrame()
for index, row in df.iterrows():
    llatitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.foursquare.com/v3/places/search?ll={llatitude},{longitude}&radius={radius}&categories={categories}&fields=name,rating,location,popularity,distance"
    result = requests.get(url, headers=headers)
    data = result.json()
    result_df = pd.json_normalize(data['results'])
    foursquaredata = pd.concat([foursquaredata, result_df], ignore_index=True)
    foursquaredata['station'] = row['name']


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [4]:
foursquareparsed = foursquaredata[['station','distance','name','location.address','rating','popularity']]

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
api_key = os.environ['yelpapi']
llatitude = ""
longitude = ''
radius = str(1000)
categories = '11124,13003'
headers = {"Accept": "application/json"}
headers['Authorization'] = api_key
foursquaredata = pd.DataFrame()
for index, row in df.iterrows():
    llatitude = row['latitude']
    longitude = row['longitude']
    url = f"https://api.foursquare.com/v3/places/search?ll={llatitude},{longitude}&radius={radius}&categories={categories}&fields=name,rating,location,popularity,distance"
    result = requests.get(url, headers=headers)
    data = result.json()
    result_df = pd.json_normalize(data['results'])
    foursquaredata = pd.concat([foursquaredata, result_df], ignore_index=True)
    foursquaredata['station'] = row['name']


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating